# Klasifikasi Citra Sederhana

## 1. Mengimport Modul

In [8]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.model import Model

ImportError: cannot import name 'set_backend' from 'keras.src.utils.backend_utils' (C:\Users\umalb\.conda\envs\percobaan\Lib\site-packages\keras\src\utils\backend_utils.py)

## 2. Menginput Gambar dan Membuatnya dalam bentuk Dataset

In [ ]:
# Contoh Penggunaan
image_dir = 'D:\Perkuliahan\Mata Kuliah\MBKM\BRIN\Data\data_kaggle\rice_leaf_diseases' #address folder gambar untuk dataset
iamge_size = (224,224) #input ukuran pada gambar
batch_size = 32 # ukuran batch awal sebesar 32

#Membuat Dataset Latihan (80% dari data total)
train_dataset = tf.keras.src.utils.image_dataset_from_directory(
    directory = image_dir, #variabel pengarahan address dataset
    validation_split = 0.2, #20% data ini digunakan untuk validasi
    subset = 'training',
    seed = 123, #Random Seed untuk pergantian (shuffling)
    image_size = image_size, # untuk menspesifikasi ukuran gambar pd variabel image_size
    batch)size = batch_size
)

#Membuat Dataset Validasi (20% dari data total)
val_dataset = tf.keras.utils.image_dataset_from_directory(
    directory = image_dir,
    validation_split = 0.2, # 20% data tersebut untuk validasi
    subset = 'validation',
    seed = 123, # Random Seed untuk Pergantian (Data acak)
    image_size = image_size,
    batch_size = batch_size
) # sebagai variabel input untuk mendapatkan nilai akurasi, loss, dll pada validasi

class_name = train_dataset.class_names
#print(class_names)

## 3. Menampilkan Dataset awal dari Sampel Acak sebanyak 20 Gambar

In [ ]:
# Menampilkan 4x4 grid pada Gambar Latihan dari Dataset Tensorflow menggunakan tf.keras.utils.image_dataset_from_directory
figur = plt.figure(figsize = (13,13))
for images, labels in train_dataset.take(1):
    for i in range (20): #menampilkan 20 sampel random gambar latihan
        ax = plt.subplot(5,4, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

In [ ]:
# Menampilkan 4x4 grid pada Gambar Validasi dari Dataset Tensorflow menggunakan tf.keras.utils.image_dataset_from_directory
figur = plt.figure(figsize = (13,13))
for images, labels in val_dataset.take(1):
    for i in range (20): #menampilkan 20 sampel random gambar latihan
        ax = plt.subplot(5,4, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

## 4. Melakukan Ekstraksi Fitur Dataset

In [ ]:
# Langkah 1 : Membuat Dataset Tensorflow dari Images Directory
def create_dataset(image_dir, image_size, batch_size):
    #Memuat dan memproses gambar dari Directory tersebut
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
    # Menggunakan ImageDataGenerator untuk memuat dan memproses gambar dari direktori.
    # Rescale 1./255 digunakan untuk mengubah rentang piksel menjadi antara 0 dan 1.
    dataset = datagen.flow_from_directory(
        directory=image_dir,
        target_size = image_size, # ukuran target gambar yang diinginkan
        batch_size = batch_size, # ukuran batch data yang akan dimuat pada setiap iterasi
        class_mode = None, # Mode kelas. None berarti kita tidak membutuhkan label kelas karena kita menggunakan dataset ini untuk prediksi, bukan klasifikasi.
        shuffle = False # shuffle=False berarti kita ingin menjaga urutan gambar sesuai dengan yang ada di direktori.
    )
    return dataset

# Langkah 2 : mendefinisikan AlexNet Model untuk Ekstraksi Fitur
def create_alexnet_model(input_shape, num_classes):
    # Buat Input Layer
    input_layer = Input(shape = input_shape)
    #Conv Layer 1
    x = Conv2D(filters=96, kernel_size=(11,11), strides = (4,4), activation = 'relu')(input_layer)
    x = MaxPooling2D(pool_size = (3,3), strides = (2,2))(x)
    #Conv Layer 2
    x = Conv2D(filters=256, kernel_size=(5,5), padding='same', activation = 'relu')
    x = MaxPooling2D(pool_size = (3,3), strides = (2,2))(x)
    #Conv Layer 3
    x = Conv2D(filters=384, kernel_size=(3,3), padding='same', activation = 'relu')
    #Conv Layer 4
    x = Conv2D(filters=384, kernel_size=(3,3), padding='same', activation = 'relu')
    #Conv Layer 5
    x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation = 'relu')
    x = MaxPooling2D(pool_size = (3,3), strides = (2,2))(x)
    #Flatten Layer
    x = Flatten()(x)
    #Fully Connected Layer 1
    x = Dense(4096, activation = 'relu')(x)
    #Fully Connected Layer 2
    x = Dense(4096, activation = 'relu')(x)
    #Output Layer
    output_layer = Dense(num_classes, activation='softmax')(x)
    #Buat Model dari input layer dan output layer
    model = tf.keras.Model(inputs=input_layer, output=output_layer)

    return model

# Langkah 3 : Mendeteksi Penyakit Daun Padi dan Menyediakan Bounding Boxes
def detect_and_bbox(dataset, model):
    predictions = model.predict(dataset, verbose =1)
    return predictions



## 5. Mendefinisikan AlexNet

In [9]:
input_shape(*image_size, 3) #bentuk daripada input gambar
num_classes = class_name # menentukan kelas dataset
# Buat model AlexNet
alexnet_model = create_alexnet_model(input_shape, num_classes)
# menampilkan ringkasan model
alexnet_model.summary() #digunakan untuk memeriksa arsitektur yang kita buat

NameError: name 'input_shape' is not defined

## 6. Mengcompile AlexNet dengan Pendekatan Aturan untuk TL

In [ ]:
# Menyusun Model AlexNet dengan Pendekatan untuk Transfer Learning
alexnet_model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics = ['accuracy']) # Loss = 'mean_squared_error', metrics = ['mse']

num_epochs = 20
# Melatih model tersebut dan mendapatkan jejak latihan
# alexnet_model.fit(train_dataset, epoch=num_epochs, validation_data = val_dataset)
history = alexnet_model.fit(train_dataset, epoch=num_epochs, validation_data=val_dataset)

## 7. Membuat Plot untuk Loss pada Latihan dan Validasi

In [ ]:
# Membuat Plot Latihan Vs Plot Validasi

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Data Accuracy')
plt.plot(history.history['val_loss'], label = 'Validation Data Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Menampilkan nilai akurasi dan loss pada nilai set validasi
loss, accuracy = alexnet_model.evaluate(val_dataset)
print(f"Validation Loss {loss}")
print(f"Validation Accuracy {accuracy}")

## 8. Mendeteksi Penyakit Daun Padi dan Menyediakan Kotak Pembatas

In [1]:
# Mendeteksi dan membuat Kotak Pembatas
predictions = detect_and_bbox(val_dataset, alexnet_model)

NameError: name 'detect_and_bbox' is not defined

## 9. Menampilkan Konfusi Matriks Data Latihan

In [ ]:
# Menampilkan Confusi Matriks dalam Data Latihan

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix

#Contoh Penggunaan
num_runs = 5 # Bilangan running rerata sampai lebih dari -n
dataset = train_dataset # dataset tensorflow
# model = tf.keras.models.load_model('model_class_13_balances_combined_ep_50.h5')  # Your trained TensorFlow model

def compute_confusion_matrix(alexnet_model, dataset, num_runs):
    all_true_labels = []
    all_pred_labels = []

    for _ in range(num_runs):
        true_labels = []
        pred_labels = []

        # Membuat Prediksi dalam dataset
        for features, labels in dataset:
            predictions = alexnet_model.predict(features)
            true_labels.extend(labels.numpy())
            pred_labels.extend(np.argmax(predictions, axis = 1))

        all_true_labels.append(true_labels)
        all_pred_labels.append(pred_labels)

    #Mengkomputasi Confusi Matrix untuk tiap Running
    all_conf_matrices = [confusion_matrix(true_labels, pred_labels) for true_labels, pred_labels in zip(all_true_labels, all_pred_labels)]

    # rerata Confusi Matrix
    avg_conf_matrix = np.mean(all_conf_matrices, axis=0)

    return avg_conf_matrix

avg_conf_matrix = compute_confusion_matrix(alexnet_model, dataset, num_runs)
print(f"Average Confusion Matrix : \n\n {avg_conf_matrix}")

# Mengubah Confusi Matrix menjadi Integers
avg_conf_matrix = avg_conf_matrix.astype(int)

class_names = class_names



## 10. Membuat HeatMap Data Training

In [ ]:
# Membuat Plot Heat Map

plt.figure(figsize = (10,8))
sns.heatmap(avg_conf_matrix, annot=True, fmt='d', cmap = 'Greens', cbar = True,
           xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## 11. Menampilkan Konfusi Matriks pada Data Validasi

In [ ]:
# Menampilkan Confusi Matriks dalam Data Latihan

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import confusion_matrix

#Contoh Penggunaan
num_runs = 5 # Bilangan running rerata sampai lebih dari -n
dataset = val_dataset # dataset tensorflow
# model = tf.keras.models.load_model('model_class_13_balances_combined_ep_50.h5')  # Your trained TensorFlow model

def compute_confusion_matrix(alexnet_model, dataset, num_runs):
    all_true_labels = []
    all_pred_labels = []

    for _ in range(num_runs):
        true_labels = []
        pred_labels = []

        # Membuat Prediksi dalam dataset
        for features, labels in dataset:
            predictions = alexnet_model.predict(features)
            true_labels.extend(labels.numpy())
            pred_labels.extend(np.argmax(predictions, axis = 1))

        all_true_labels.append(true_labels)
        all_pred_labels.append(pred_labels)

    #Mengkomputasi Confusi Matrix untuk tiap Running
    all_conf_matrices = [confusion_matrix(true_labels, pred_labels) for true_labels, pred_labels in zip(all_true_labels, all_pred_labels)]

    # rerata Confusi Matrix
    avg_conf_matrix = np.mean(all_conf_matrices, axis=0)

    return avg_conf_matrix

avg_conf_matrix = compute_confusion_matrix(alexnet_model, dataset, num_runs)
print(f"Average Confusion Matrix : \n\n {avg_conf_matrix}")

# Mengubah Confusi Matrix menjadi Integers
avg_conf_matrix = avg_conf_matrix.astype(int)

class_names = class_names



## 12. Membuat HeatMap Data Validasi

In [2]:
# Membuat Plot Heat Map

plt.figure(figsize = (10,8))
sns.heatmap(avg_conf_matrix, annot=True, fmt='d', cmap = 'Greens', cbar = True,
           xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

NameError: name 'plt' is not defined

## 13. Menampilkan Laporan Klasifikasi Data Training dan Validasi

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
import pandas as pd

# Contoh Penggunaan
num_runs = 5
dataset = train_dataset


def compute_classification_report(model, dataset):
    true_labels = []
    pred_labels = []

    # membuat prediksi pada dataset
    for features, labels in dataset:
        predictions = alexnet_model.predict(features)
        true_labels.extend(labels.numpy())
        pred_labels.extend(np.argmax(predictions, axis=1))

    # Mengkomputasikan laporan Klasifikasi
    classification_report_str = classification_report(true_labels, pred_labels)
    return classification_report_str


# Mengkomputasi Hasil Laporan Klasifikasi
classification_report_result = compute_classification_report(alexnet_model, dataset)

# Menampilkan Laporan Klasifikasi 
print(f"Classification Report :\n\n {classification_report_result}")


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
import pandas as pd

# Contoh Penggunaan
num_runs = 5
dataset = val_dataset


def compute_classification_report(model, dataset):
    true_labels = []
    pred_labels = []

    # membuat prediksi pada dataset
    for features, labels in dataset:
        predictions = alexnet_model.predict(features)
        true_labels.extend(labels.numpy())
        pred_labels.extend(np.argmax(predictions, axis=1))

    # Mengkomputasikan laporan Klasifikasi
    classification_report_str = classification_report(true_labels, pred_labels)
    return classification_report_str


# Mengkomputasi Hasil Laporan Klasifikasi
classification_report_result = compute_classification_report(alexnet_model, dataset)

# Menampilkan Laporan Klasifikasi 
print(f"Classification Report :\n\n {classification_report_result}")
